In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from ResNet import ResNet, ResNet101

In [10]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [11]:
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test, batch_size=128,shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = ResNet101(10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [15]:
EPOCHS = 1
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    for i, inp in enumerate(trainloader):
        inputs, labels = inp
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%1 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')

Loss [1, 1](epoch, minibatch):  0.07317168712615967
Loss [1, 2](epoch, minibatch):  0.052230825424194334
Loss [1, 3](epoch, minibatch):  0.04962517261505127
Loss [1, 4](epoch, minibatch):  0.026245033740997313
Loss [1, 5](epoch, minibatch):  0.03653975486755371
Loss [1, 6](epoch, minibatch):  0.052273130416870116
Loss [1, 7](epoch, minibatch):  0.06685672283172607
Loss [1, 8](epoch, minibatch):  0.02310443639755249
Loss [1, 9](epoch, minibatch):  0.07231516361236573
Loss [1, 10](epoch, minibatch):  0.03223624467849731
Loss [1, 11](epoch, minibatch):  0.027728211879730225
Loss [1, 12](epoch, minibatch):  0.02800156831741333
Loss [1, 13](epoch, minibatch):  0.03810319900512695
Loss [1, 14](epoch, minibatch):  0.029915854930877686
Loss [1, 15](epoch, minibatch):  0.02435231685638428
Loss [1, 16](epoch, minibatch):  0.029880506992340086
Loss [1, 17](epoch, minibatch):  0.032362701892852785
Loss [1, 18](epoch, minibatch):  0.023562648296356202
Loss [1, 19](epoch, minibatch):  0.030006036758

KeyboardInterrupt: 

In [16]:
correct = 0
total = 0

with torch.no_grad():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

Accuracy on 10,000 test images:  13.120000000000001 %
